In [23]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from bs4 import BeautifulSoup as bs

In [29]:
# 20230705171919_관광지별 성_연령 발문 집중률 조회.csv
place_name = ['경주골굴사', '경주김유신묘', '경주동궁과월지'
,'경주동부사적지대','경주무열왕릉','경주문무대왕릉'
,'경주불국사', '경주석굴암', '경주신문왕릉', '경주월성(반월성)'
, '경주첨성대', '경주금곡사', '경주기림사', '경주백률사', '경주분황사' 
,'경주연지암', '경주주사암']

# place_number = {
#     '경주골굴사':['02089','02090'], '경주김유신묘':['01025','01036'], 
#     '경주동궁과월지':['01225', '01237'], '경주동부사적지대':['01279','01280'], 
#     '경주무열왕릉':['01194','01195'], '경주문무대왕릉':['02056','02057'],
#     '경주불국사':['01160','01161'], '경주석굴암':['01199','01199'], 
#     '경주신문왕릉':['01081','01082'], '경주월성(반월성)':['01237','01238'], 
#     '경주첨성대':['01279','01280'], '경주금곡사':['05379','05380'],
#     '경주기림사':['02021','02021'], '경주백률사':['01228','01229'],
#     '경주분황사':['01157','01158'],'경주연지암':['05423','05426'], 
#     '경주주사암':['03121','03122']
# } 
# place_number = ['02089','02090','01025','01036','01225', '01237','01279','01280',
#                '01194','01195','02056','02057','01160','01161','01199',
#                '01081','01082','01238','01280','05379','05380',
#                '02021','01228','01229','01157','01158','14104','14105']
# } 
place_number = ['02021','01199']

In [24]:
path='../../driver/chromedriver.exe'
driver = webdriver.Chrome(service=Service(path))

In [25]:
# 네이버 지도 접속
driver.get('https://map.naver.com/')
time.sleep(3)

### 버스 정보 받아오기

In [26]:
def bus_info(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 정류장 이름 리스트 저장
    station_name = []
    d = soup.select('div.route_area>ul>li>a>span>strong')
    for x in d:
        station_name.append(x.text)

    time.sleep(3)

    # 정류장 고유 번호 리스트 저장
    station_number = []
    e = soup.select('div.route_area>ul>li>a>div.route_box')
    for x in e:
        station_number.append(x.text)
    time.sleep(3)

    bus_data = {'버스노선':station_name, '정류장번호':station_number}

    return bus_data

In [37]:
# 배너 '버스' 클릭
element = driver.find_element(By.CLASS_NAME,"link_navbar.bus")
element.click()
time.sleep(5)

# 버스 정류장 클릭
busstop_button = driver.find_element(By.CLASS_NAME,"link_tab.busstop")
busstop_button.click()
time.sleep(3)

for i in range(len(place_number)):
    
    # 버스 정류장 검색 클릭
    busstop_search = driver.find_element(By.ID,"input_search1689055109366")
####   busstop_search ID 말고 다른 방법 찾아보기

    time.sleep(2)
    # place_num 입력
    searh_name = place_number[i]
    busstop_search.send_keys(searh_name)
    busstop_search.send_keys(Keys.ENTER)
    time.sleep(3)

    # 전체지역 -> 경상북도 클릭
    location_button = driver.find_element(By.CLASS_NAME,"btn_option")
    location_button.click()
    time.sleep(3)
    geongbuk_button = driver.find_elements(By.CLASS_NAME, 'link_select')
    for i in range(len(geongbuk_button)):
        if geongbuk_button[i].text == '경상북도':
            geongbuk_button[i].click()
    time.sleep(4)

    # 버스 정보 클릭
    location_button = driver.find_element(By.CLASS_NAME,"search_result_content_area")
    location_button.click()
    time.sleep(4)

#     # 정류장의 버스 개수 세기
#     count_bus = driver.find_elements(By.CLASS_NAME, 'item_station.ng-star-inserted')
#     time.sleep(4)

    # 버스 순서대로 클릭
    bus_info = driver.find_element(By.CLASS_NAME, "item_station.ng-star-inserted")
    bus_info.click()
    time.sleep(5)

    # 운행시간, 배차간격(첫차) 가져오기
    first = driver.find_element(By.CLASS_NAME,"item_bus_info.ng-star-inserted")
    first_time = first.find_element(By.CLASS_NAME,"value").text[3:]
    time.sleep(5)

    per_time = driver.find_element(By.CLASS_NAME,"value.ng-star-inserted").text
    time.sleep(5)

    # 버스 번호 받아오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    bus_num = soup.select('div.title_wrap_box>div.title_box>strong')[0].text

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 정류장 이름 리스트 저장
    station_name = []
    d = soup.select('div.route_area>ul>li>a>span>strong')
    for x in d:
        station_name.append(x.text)

    time.sleep(5)

    # 정류장 고유 번호 리스트 저장
    station_number = []
    e = soup.select('div.route_area>ul>li>a>div.route_box')
    for x in e:
        station_number.append(x.text)
    time.sleep(5)

#         bus_data = {'버스노선':station_name, '정류장번호':station_number}
#     print(bus_data)
    # 데이터프레임 저장
    df = pd.DataFrame()

    df['버스노선'] = station_name
    df['정류장번호'] = station_number
    df['버스번호'] = bus_num
    df['도착시간'] = first_time
    df['배차간격'] = per_time

    df.to_excel(bus_num+f'_{j}_{searh_name}'+".xlsx")
    time.sleep(5)
    # 뒤로가기 버튼 누르기
    back_button = driver.find_element(By.CLASS_NAME,"btn_back")
    back_button.click()
    time.sleep(5)

    # 도착지 이전 입력값 지우기
    busstop_search.send_keys(Keys.CONTROL + "A")
    busstop_search.send_keys(Keys.DELETE)
    time.sleep(3)

In [35]:
driver.find_elements(By.CLASS_NAME, "list_station")
   

[<selenium.webdriver.remote.webelement.WebElement (session="863c2d882d2e8d66ec5be15ab314e0fc", element="EA5CAE57BBA1BFA5487986DB9E690A66_element_582")>]

In [19]:
# 배너 '버스' 클릭
element = driver.find_element(By.CLASS_NAME,"link_navbar.bus")
element.click()
time.sleep(5)

# 버스 정류장 클릭
busstop_button = driver.find_element(By.CLASS_NAME,"link_tab.busstop")
busstop_button.click()
time.sleep(3)

for i in range(len(place_number)):
    
    # 버스 정류장 검색 클릭
    busstop_search = driver.find_element(By.ID,"input_search1689033438137")
####   busstop_search ID 말고 다른 방법 찾아보기

    time.sleep(2)
    # place_num 입력
    searh_name = place_number[i]
    busstop_search.send_keys(searh_name)
    busstop_search.send_keys(Keys.ENTER)
    time.sleep(3)

    # 전체지역 -> 경상북도 클릭
    location_button = driver.find_element(By.CLASS_NAME,"btn_option")
    location_button.click()
    time.sleep(3)
    geongbuk_button = driver.find_elements(By.CLASS_NAME, 'link_select')
    for i in range(len(geongbuk_button)):
        if geongbuk_button[i].text == '경상북도':
            geongbuk_button[i].click()
    time.sleep(4)

    # 버스 정보 클릭
    location_button = driver.find_element(By.CLASS_NAME,"search_result_content_area")
    location_button.click()
    time.sleep(4)

    # 정류장의 버스 개수 세기
    count_bus = driver.find_elements(By.CLASS_NAME, 'item_station.ng-star-inserted')
    time.sleep(4)
    
    for j in range(len(count_bus)):
        # 버스 순서대로 클릭
        bus_info = driver.find_elements(By.CLASS_NAME, "item_station.ng-star-inserted")
        bus_info[j].click()
        time.sleep(5)

        # 운행시간, 배차간격(첫차) 가져오기
        first = driver.find_element(By.CLASS_NAME,"item_bus_info.ng-star-inserted")
        first_time = first.find_element(By.CLASS_NAME,"value").text[3:]
        time.sleep(5)

        per_time = driver.find_element(By.CLASS_NAME,"value.ng-star-inserted").text
        time.sleep(5)

        # 버스 번호 받아오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        bus_num = soup.select('div.title_wrap_box>div.title_box>strong')[0].text

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # 정류장 이름 리스트 저장
        station_name = []
        d = soup.select('div.route_area>ul>li>a>span>strong')
        for x in d:
            station_name.append(x.text)

        time.sleep(5)

        # 정류장 고유 번호 리스트 저장
        station_number = []
        e = soup.select('div.route_area>ul>li>a>div.route_box')
        for x in e:
            station_number.append(x.text)
        time.sleep(5)

#         bus_data = {'버스노선':station_name, '정류장번호':station_number}
    #     print(bus_data)
        # 데이터프레임 저장
        df = pd.DataFrame()

        df['버스노선'] = station_name
        df['정류장번호'] = station_number
        df['버스번호'] = bus_num
        df['도착시간'] = first_time
        df['배차간격'] = per_time
        
        df.to_excel(bus_num+f'_{j}_{searh_name}'+".xlsx")
        time.sleep(5)
        # 뒤로가기 버튼 누르기
        back_button = driver.find_element(By.CLASS_NAME,"btn_back")
        back_button.click()
        time.sleep(5)

    # 도착지 이전 입력값 지우기
    busstop_search.send_keys(Keys.CONTROL + "A")
    busstop_search.send_keys(Keys.DELETE)
    time.sleep(3)

In [ ]:
df = pd.read_excel('./좌석1000.xlsx')
df

In [ ]:
count_bus = driver.find_elements(By.CLASS_NAME, 'item_station.ng-star-inserted')
len(count_bus)

In [ ]:
# ap = pd.read_excel('./좌석150_6_01237.xlsx')
# ap

In [ ]:
# a = soup.select('div.title_wrap_box>div.title_box>strong')[0].text
# a

In [ ]:
# # 뒤로가기 버튼

# back_button = driver.find_element(By.CLASS_NAME,"btn_back")
# back_button.click()
# time.sleep(3)

In [ ]:
# # 정류장의 버스 개수 세기
# # b = soup.select('div.station_area')
# c = driver.find_elements(By.CLASS_NAME, 'item_station.ng-star-inserted')
# len(c)

In [ ]:
#  bus_info[2]

In [1]:
import pandas as pd

In [6]:
lst = []
for i in range(3):
    df = pd.DataFrame()
    df['test'] = i
    lst.append(df)
lst[2]

,test


In [18]:
lst = []
for i in range(3):
    ap = pd.DataFrame({'ID':[20190103, 20190222, 20190531],
                                 'name':['Kim', 'Lee', 'Jeong'],
                                 'class':['H', 'W', 'S']})
    lst.append(ap)
lst[1]

,ID,name,class
0,20190103,Kim,H
1,20190222,Lee,W
2,20190531,Jeong,S
